In [ ]:
# !pip install FlagEmbedding pymilvus "pymilvus[model]"

# 사전 임베딩

{'AirConditioner': '에어컨은 냉방, 난방, 제습, 송풍 등 다양한 모드로 실내 온도와 습도를 조절하는 기기입니다. 전원을 켜거나 끌 수 있으며, 원하는 모드로 변경하거나 온도를 설정할 수 있습니다. 자동 모드를 통해 상황에 따라 최적의 상태를 유지할 수도 있습니다.; 에어컨 켜 줘;에어컨 꺼 줘;에어컨 온도를 24도로 설정해 줘;시원하게 해 줘;에어컨 바람 세게 해 줘;제습 모드로 바꿔 줘;에어컨을 자동 모드로 설정해 줘;바람만 나오게 해 줘;오전 10시에 에어컨을 켜 줘;요즘 너무 습해서 제습 모드로 계속 유지해 줘;냉방기 전원을 꺼 줘;에어컨디셔너를 난방 모드로 바꿔 줘;거실 에어컨을 수동 모드로 변경해 줘;실내 냉방기 바람을 약하게 해 줘;바람 나오는 기기 온도를 28도로 맞춰 줘;벽걸이 에어컨을 조용 모드로 바꿔 줘;냉방기를 오후 6시에 자동으로 꺼 줘;에어컨 온도를 매일 밤 11시에 26도로 설정해 줘;에어컨의 모드를 송풍으로 전환해 줘;바람 세기 중간으로 설정해 줘;에어컨을 무풍 모드로 바꿔 줘;거실 냉방기 온도 낮춰 줘;벽걸이형 에어컨을 강풍으로 설정해 줘;온도가 올라가면 자동으로 시원한 바람 나오게 해 줘;실내가 너무 더우면 냉방기 바람 세게 해 줘', 'AirPurifier': '공기청정기는 실내 공기 중의 먼지, 미세먼지, 냄새 등을 줄여 쾌적한 환경을 만들어 주는 기기입니다. 전원을 켜고 끌 수 있으며, 팬 속도를 자동, 낮음, 중간, 높음, 수면, 조용, 무풍 등 다양한 모드로 조절할 수 있습니다.; 공기청정기 켜 줘;공기청정기 꺼 줘;공기청정기 전원 꺼 줘;공기청정기 전원 켜 줘;공기청정기 풍량을 자동으로 설정해 줘;공기청정기 풍량을 강하게 해 줘;공기청정기 풍량을 약하게 해 줘;공기청정기를 무풍 모드로 바꿔 줘;공기청정기를 수면 모드로 설정해 줘;공기청정기 팬을 조용 모드로 바꿔 줘;공기청정기 팬을 중간으로 설정해 줘;공기청정기 팬을 꺼 줘;공기청정기에서 설정할 수 있는 모드 알려 줘;공기청정기 모드를 수면에서 자동으로

In [ ]:
from FlagEmbedding import BGEM3FlagModel
import numpy as np
import json, pickle

# 모델 초기화 (CPU/GPU 설정)
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# with open("../ServiceExtraction/integration/0.1.5_embedding_all.json", "r") as f:
#     services = json.load(f)

with open("../ServiceExtraction/integration/0.1.6_natural.json", "r") as f:
    services = {}
    data = json.load(f)
    for device_id, device_info in data.items():
        services[device_id] = f"{device_info["info"]}; {";".join(device_info["examples"])}"
    # print(services)

keys = list(services.keys())
texts = list(services.values())

# 임베딩 생성 (배치 처리)
batch_size = 32
dense_embeddings = []
sparse_embeddings = []
colbert_embeddings = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    outputs = model.encode(
        batch, 
        return_dense=True,
        return_sparse=True,
        return_colbert_vecs=True  # ColBERT 활성화
    )
    dense_embeddings.extend(outputs['dense_vecs'])
    sparse_embeddings.extend(outputs['lexical_weights'])
    colbert_embeddings.extend(outputs['colbert_vecs'])

# ColBERT 벡터 저장 전처리
def process_colbert(embeddings):
    """3D 배열을 저장 가능한 형태로 변환"""
    return [emb.astype(np.float16) for emb in embeddings]  # 절반의 저장 공간 절약

processed_colbert = process_colbert(colbert_embeddings)

# 변환 함수 확장
def convert_to_serializable(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.float16):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(i) for i in obj]
    else:
        return obj

# 저장
np.save('./embedding_result/dense_embeddings.npy', np.array(dense_embeddings))
# ColBERT 벡터 압축 저장
with open('./embedding_result/colbert_embeddings.pkl', 'wb') as f:
    pickle.dump(processed_colbert, f)

# float32 → float 로 강제 변환
def convert_to_serializable(obj):
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(i) for i in obj]
    elif isinstance(obj, np.float32):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

serializable_sparse = convert_to_serializable(sparse_embeddings)

with open('./embedding_result/sparse_embeddings.json', 'w') as f:
    json.dump(serializable_sparse, f, indent=2)
    
# 메타데이터 저장 (ColBERT 정보 추가)
metadata = {
    'keys': keys,
    'texts': texts,
    'colbert_shapes': [emb.shape for emb in processed_colbert]  # 원본 형태 정보
}
with open('./embedding_result/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 165782.77it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


# 호출

##  dense_vecs (문장 전체 평균)만 활용

In [3]:
import numpy as np
import json, pickle
from sklearn.metrics.pairwise import cosine_similarity

# 임베딩 데이터 로드
dense_embeddings = np.load('./embedding_result/dense_embeddings.npy')
with open('./embedding_result/sparse_embeddings.json') as f:
    sparse_embeddings = json.load(f)
with open('./embedding_result/metadata.json') as f:
    metadata = json.load(f)

# 모델 초기화 (CPU 전용)
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 149796.57it/s]


In [10]:

def recommend_services(query, top_k=10):
    # 쿼리 임베딩 생성
    query_dense = model.encode([query], return_dense=True)['dense_vecs'][0]
    
    # 유사도 계산
    dense_scores = cosine_similarity([query_dense], dense_embeddings)[0]
    
    # 상위 K개 결과 추출
    top_indices = np.argsort(dense_scores)[-top_k:][::-1]
    
    return [
        {
            'key': metadata['keys'][i],
            'text': metadata['texts'][i],
            'score': float(dense_scores[i])
        }
        for i in top_indices
    ]

# 사용자 입력 처리
user_query = "기온이 30도 이상이면 커튼을 닫고 에어컨을 틀어줘"
results = recommend_services(user_query)

# 결과 출력
print(f"추천 서비스 (쿼리: '{user_query}'):")
for idx, result in enumerate(results, 1):
    print(f"{idx}. {result['key']} (유사도: {result['score']:.4f})")
    print(f"   내용: {result['text'][:50]}...")

추천 서비스 (쿼리: '기온이 30도 이상이면 커튼을 닫고 에어컨을 틀어줘'):
1. AirConditioner (유사도: 0.6530)
   내용: 에어컨은 냉방, 난방, 제습, 송풍 등 다양한 모드로 실내 온도와 습도를 조절하는 기기입니...
2. Curtain (유사도: 0.6228)
   내용: 커튼은 열고 닫고 멈추는 기능을 가진 장치로, 햇빛 조절이나 사생활 보호를 위해 사용됩니다...
3. Blind (유사도: 0.5736)
   내용: 블라인드(커튼)는 창문을 덮거나 열어 햇빛의 양을 조절하고 사생활을 보호하는 장치입니다. ...
4. TemperatureSensor (유사도: 0.5592)
   내용: 온도 센서는 현재 환경의 온도를 측정하여 수치로 제공합니다. 주로 특정 온도 조건에 따라 ...
5. Humidifier (유사도: 0.5570)
   내용: 가습기는 실내 습도를 조절해주는 장치로, 전원을 켜고 끌 수 있으며 자동, 약풍, 중간, ...
6. Fan (유사도: 0.5387)
   내용: 선풍기나 환풍기 등의 팬 장치는 전원을 켜고 끌 수 있으며, 풍속을 RPM이나 퍼센트로 조...
7. WeatherProvider (유사도: 0.5333)
   내용: 날씨 제공 장치는 현재 기온, 습도, 기압, 미세먼지 수치, 날씨 상태 등을 바탕으로 자동...
8. DoorLock (유사도: 0.5332)
   내용: 도어락은 문을 원격으로 열고 닫을 수 있는 기기로, 현재 문이 열려 있는지 닫혀 있는지도 ...
9. AirPurifier (유사도: 0.5262)
   내용: 공기청정기는 실내 공기 중의 먼지, 미세먼지, 냄새 등을 줄여 쾌적한 환경을 만들어 주는 ...
10. Window (유사도: 0.5209)
   내용: 창문은 열림, 닫힘, 또는 상태를 알 수 없는 상태로 존재하며, 환기나 보안, 환경 조건에...


## colbert_vecs를 활용한 dense token-level 다중 검색

In [83]:
import numpy as np
import json, pickle
from sklearn.metrics.pairwise import cosine_similarity
from FlagEmbedding import BGEM3FlagModel

# 모델 및 데이터 초기화
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)  # CPU 환경

# 임베딩 데이터 로드 (ColBERT 추가)
dense_embeddings = np.load('./embedding_result/dense_embeddings.npy')
# colbert_data = np.load('./embedding_result/colbert_embeddings.npz', allow_pickle=True)
# colbert_embeddings = [emb.astype(np.float32) for emb in colbert_data['colbert']]  # float32로 변환
with open('./embedding_result/colbert_embeddings.pkl', 'rb') as f:
    colbert_embeddings = pickle.load(f)

with open('./embedding_result/sparse_embeddings.json') as f:
    sparse_embeddings = json.load(f)
    
with open('./embedding_result/metadata.json') as f:
    metadata = json.load(f)



Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 149618.45it/s]


In [84]:
# ColBERT 유사도 계산 함수
# 1) 평균 맥스심 점수
def colbert_maxsim(query_vec, doc_vecs):
    """
    query_vec: [query_tokens, dim]
    doc_vecs: [doc_tokens, dim]
    """
    sim_matrix = cosine_similarity(query_vec, doc_vecs)
    return np.max(sim_matrix, axis=1).mean()  

# 2) Softmax MaxSim
def colbert_softmax_maxsim(query_vec, doc_vecs, temperature=0.05):
    sim_matrix = cosine_similarity(query_vec, doc_vecs)
    max_sim = np.max(sim_matrix, axis=1)
    weights = np.exp(max_sim / temperature)
    weights /= np.sum(weights)
    return np.sum(weights * max_sim)

# 하이브리드 추천 함수
def hybrid_recommend(query, top_k=10, weights=(0.6, 0.3, 0.1)):
    # 쿼리 임베딩 생성
    query_emb = model.encode(
        [query], 
        return_dense=True,
        return_sparse=True,
        return_colbert_vecs=True
    )
    
    # 각 유사도 계산
    dense_scores = cosine_similarity([query_emb['dense_vecs'][0]], dense_embeddings)[0]
    
    sparse_scores = []
    query_weights = query_emb['lexical_weights'][0]  # dict

    for doc_weights in sparse_embeddings:  # 문서별 sparse dict
        score = sum(query_weights.get(token, 0) * doc_weights.get(token, 0) for token in query_weights)
        sparse_scores.append(score)
    
    colbert_scores = [
        colbert_softmax_maxsim(query_emb['colbert_vecs'][0], doc_emb)
        for doc_emb in colbert_embeddings
    ]
    
    # 점수 정규화 및 결합
    max_score = max(dense_scores.max(), 1e-6)
    combined_scores = (
        weights[0] * dense_scores/max_score +
        weights[1] * np.array(sparse_scores) +
        weights[2] * np.array(colbert_scores)
    )
    
    # 상위 K개 추출
    top_indices = np.argsort(combined_scores)[-top_k:][::-1]
    
    return [{
        'key': metadata['keys'][i],
        'text': metadata['texts'][i],
        'dense_score': float(dense_scores[i]),
        'sparse_score': float(sparse_scores[i]),
        'colbert_score': float(colbert_scores[i]),
        'combined_score': float(combined_scores[i])
    } for i in top_indices]


In [87]:
# 추천 실행
results = hybrid_recommend("기온이 30도 이상이면 커튼을 닫고 에어컨을 틀어줘", 
                           top_k=10)

# 결과 출력
print("추천 서비스:")
for idx, item in enumerate(results, 1):
    print(f"{idx}. {item['key']}")
    print(f"   종합 점수: {item['combined_score']:.4f}")
    print(f"   Dense: {item['dense_score']:.3f}, Sparse: {item['sparse_score']:.3f}, ColBERT: {item['colbert_score']:.3f}")


추천 서비스:
1. Curtain_curtain_close
   종합 점수: 0.6690
   Dense: 0.654, Sparse: 0.000, ColBERT: 0.690
2. Curtain_curtain_open
   종합 점수: 0.6355
   Dense: 0.616, Sparse: 0.000, ColBERT: 0.698
3. AirConditioner_switch_on
   종합 점수: 0.6186
   Dense: 0.603, Sparse: 0.000, ColBERT: 0.650
4. AirConditioner_switch_off
   종합 점수: 0.6132
   Dense: 0.597, Sparse: 0.000, ColBERT: 0.651
5. Shade_windowShade_close
   종합 점수: 0.5943
   Dense: 0.578, Sparse: 0.000, ColBERT: 0.634
6. Window_windowControl_close
   종합 점수: 0.5942
   Dense: 0.576, Sparse: 0.000, ColBERT: 0.652
7. AirConditioner_airConditionerMode_setTemperature
   종합 점수: 0.5937
   Dense: 0.576, Sparse: 0.000, ColBERT: 0.652
8. Valve_valve_close
   종합 점수: 0.5864
   Dense: 0.571, Sparse: 0.000, ColBERT: 0.624
9. AirConditioner_switch_toggle
   종합 점수: 0.5860
   Dense: 0.569, Sparse: 0.000, ColBERT: 0.637
10. Fan_switch_off
   종합 점수: 0.5792
   Dense: 0.561, Sparse: 0.000, ColBERT: 0.645
